<a href="https://colab.research.google.com/github/Cincin3112/Cross-lingual-NLI-for-Thai/blob/main/wangchanBERTa_Thai_NLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install & import

In [1]:
#check GPU 
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Dec  7 09:14:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 21.5 MB/s 
     |████████████████████████████████| 7.6 MB 61.2 MB/s 
     |████████████████████████████████| 182 kB 57.4 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 32.7 MB/s 
     |████████████████████████████████| 212 kB 76.9 MB/s 
     |████████████████████████████████| 132 kB 79.8 MB/s 
     |████████████████████████████████| 127 kB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 20.1 MB/s 


In [5]:
from transformers import (
    AutoTokenizer, 
    AutoModel,
    AutoModelForSequenceClassification, 
    AutoConfig,
    Trainer, 
    TrainingArguments,
)
from datasets import (
    Dataset,
    DatasetDict,
    load_dataset,
    Features, Sequence, ClassLabel, Value
)

import pandas as pd

# load model

In [79]:
model_name = 'airesearch/wangchanberta-base-att-spm-uncased'
config = AutoConfig.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--airesearch--wangchanberta-base-att-spm-uncased/snapshots/abe46f39cf2c911a6ad5ec8299bdf7503edc95e4/config.json
Model config CamembertConfig {
  "_name_or_path": "airesearch/wangchanberta-base-att-spm-uncased",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_head": 12,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding

In [80]:
from datasets import list_metrics, load_metric
list_metrics()

['accuracy',
 'bertscore',
 'bleu',
 'bleurt',
 'brier_score',
 'cer',
 'chrf',
 'code_eval',
 'comet',
 'competition_math',
 'coval',
 'cuad',
 'exact_match',
 'f1',
 'frugalscore',
 'glue',
 'google_bleu',
 'indic_glue',
 'mae',
 'mahalanobis',
 'mape',
 'mase',
 'matthews_correlation',
 'mauve',
 'mean_iou',
 'meteor',
 'mse',
 'nist_mt',
 'pearsonr',
 'perplexity',
 'poseval',
 'precision',
 'recall',
 'rl_reliability',
 'roc_auc',
 'rouge',
 'sacrebleu',
 'sari',
 'seqeval',
 'smape',
 'spearmanr',
 'squad',
 'squad_v2',
 'super_glue',
 'ter',
 'trec_eval',
 'wer',
 'wiki_split',
 'xnli',
 'xtreme_s',
 'Drunper/metrica_tesi',
 'Felipehonorato/my_metric',
 'GMFTBY/dailydialog_evaluate',
 'GMFTBY/dailydialogevaluate',
 'JP-SystemsX/nDCG',
 'KevinSpaghetti/accuracyk',
 'NCSOFT/harim_plus',
 'NikitaMartynov/spell-check-metric',
 'NimaBoscarino/weat',
 'Ochiroo/rouge_mn',
 'Vertaix/vendiscore',
 'Viona/infolm',
 'Vlasta/pr_auc',
 'abdusah/aradiawer',
 'abidlabs/mean_iou',
 'abidlabs/me

In [81]:
load_metric('accuracy')

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

# load data

In [62]:
#for thai dataset
!gdown 1DTktDhCzaWH5ZqXxR1F5ECCw81KxalIW

Downloading...
From: https://drive.google.com/uc?id=1DTktDhCzaWH5ZqXxR1F5ECCw81KxalIW
To: /content/THtwitter_NLI.csv
100% 492k/492k [00:00<00:00, 84.4MB/s]


In [63]:
df = pd.read_csv('THtwitter_NLI.csv')

In [64]:
#แปลง label เป็น index 
labels = list(set(df['label'].to_list()))
label_to_idx = {label:i for i,label in enumerate(labels)}
df['label'] = df['label'].apply(lambda x: label_to_idx[x])

label_list = [0] * len(label_to_idx)
for label in label_to_idx:
  label_list[label_to_idx[label]] = label 

In [65]:
print(label_to_idx)

{'contradiction': 0, 'neutral': 1, 'entailment': 2}


In [66]:
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(cls_token, sep_token, pad_token, unk_token)
print(tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id,  tokenizer.unk_token_id)

<s> </s> <pad> <unk>
5 6 1 3


In [67]:
#split train, dev, test set
from sklearn.model_selection import train_test_split

df_train, the_rest = train_test_split(df, train_size=0.7)
df_dev, df_test = train_test_split(the_rest, test_size=0.5)

len(df_train), len(df_dev), len(df_test)

(739, 158, 159)

In [68]:
df_test.groupby('label').count()

,premise,hypothesis
label,,
0,51,51
1,55,55
2,53,53


#tokenize data


In [69]:
d = DatasetDict({'train': Dataset.from_pandas(df_train), 
                 'dev': Dataset.from_pandas(df_dev),
                 'test': Dataset.from_pandas(df_test)})
d = d.remove_columns(['__index_level_0__'])

d

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 739
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 158
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label'],
        num_rows: 159
    })
})

In [70]:
#tokenizer 
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['premise'], 
                                examples['hypothesis'],
                                add_special_tokens = True,
                                truncation=True,
                                is_split_into_words=False,
                                max_length=512)
                                 #pad_to_max_length = True)
    return tokenized_inputs

tokenized_datasets = d.map(tokenize, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [71]:
print(tokenized_datasets['train'][0])

{'premise': 'มิติใหม่ของห้องพัก 1 คืน 280 บาทเท่านั้น รับประกันความหลอน 100 % ขนหัวลุกเลยมึง น่าสะพรึงกลัวมาก !!!!!', 'hypothesis': 'ห้องพักราคาแพง', 'label': 0, 'input_ids': [5, 10, 3027, 8370, 4987, 10, 59, 10, 561, 10, 18201, 10, 177, 268, 10, 3199, 63, 8840, 10, 700, 10, 2024, 10, 1381, 303, 3085, 48, 521, 12677, 1633, 660, 13301, 632, 82, 10, 1000, 1525, 6, 6, 10, 4987, 8918, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [72]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 739
    })
    dev: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 158
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'attention_mask'],
        num_rows: 159
    })
})

In [73]:
#drop column ที่ไม่ใช่ทิ้ง
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer,
                                        padding=True, pad_to_multiple_of=8)

tokenized_datasets = tokenized_datasets.remove_columns(["premise"])
tokenized_datasets = tokenized_datasets.remove_columns(["hypothesis"])
#tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['label', 'input_ids', 'attention_mask']

# train  model

In [82]:
args = TrainingArguments(
    output_dir='cross-lingual-Thai-NLI', 
    learning_rate=4e-5,
    per_device_train_batch_size=16, 
    per_device_eval_batch_size=16,
    num_train_epochs=10, 
    load_best_model_at_end=True, 
    metric_for_best_model='accuracy',
    evaluation_strategy='steps', 
    eval_steps=50,
    #save_strategy='epoch'    
)

#metric.compute(predictions=preds, references=predictions.label_ids)
import numpy as np

def compute_metrics(eval_preds):
  metric = load_metric("accuracy")
  logits, labels = eval_preds 
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["dev"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 739
  Num Epochs = 10
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 24
  Gradient Accumulation steps = 1
  Total optimization steps = 310
  Number of trainable parameters = 105246723


Step,Training Loss,Validation Loss,Accuracy
50,No log,1.162614,0.253165
100,No log,1.203762,0.240506
150,No log,1.234891,0.240506
200,No log,1.148651,0.284810
250,No log,1.206024,0.253165
300,No log,1.229173,0.278481


***** Running Evaluation *****
  Num examples = 158
  Batch size = 24
***** Running Evaluation *****
  Num examples = 158
  Batch size = 24
***** Running Evaluation *****
  Num examples = 158
  Batch size = 24
***** Running Evaluation *****
  Num examples = 158
  Batch size = 24
***** Running Evaluation *****
  Num examples = 158
  Batch size = 24
***** Running Evaluation *****
  Num examples = 158
  Batch size = 24


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=310, training_loss=1.0762489564957158, metrics={'train_runtime': 126.1846, 'train_samples_per_second': 58.565, 'train_steps_per_second': 2.457, 'total_flos': 342742226915088.0, 'train_loss': 1.0762489564957158, 'epoch': 10.0})

In [83]:
predictions = trainer.predict(tokenized_datasets['test'])
preds = np.argmax(predictions.predictions, axis=-1)

***** Running Prediction *****
  Num examples = 159
  Batch size = 24


In [84]:
predicted_labels = [labels[x] for x in preds]
gold = [labels[x] for x in predictions.label_ids]

In [85]:
from sklearn.metrics import classification_report
print(classification_report(gold, predicted_labels))

               precision    recall  f1-score   support

contradiction       0.20      0.18      0.19        51
   entailment       0.25      0.42      0.31        53
      neutral       0.21      0.11      0.14        55

     accuracy                           0.23       159
    macro avg       0.22      0.23      0.22       159
 weighted avg       0.22      0.23      0.22       159

